In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [20]:
import os
import glob
import numpy as np
import pylab as plt

from processPTX import getRawXYZ, neighborFilter, radialFilter, rotateXYaboutZ

In [3]:
n = 512
xOffset = -50; yOffset=-8; rot=80.; radius=45.
path = "/home/sandboxes/pmargani/LASSI/data/9oct2019/"

In [4]:
sn = 333
re = '{0}/{1}_*.ptx'.format(path, sn)
fn = glob.glob(re)[0]

In [5]:
%%time
with open(fn, 'r') as f:
    ls = f.readlines()

CPU times: user 806 ms, sys: 578 ms, total: 1.38 s
Wall time: 1.39 s


In [6]:
%%time
x, y, z, i = getRawXYZ(ls, sampleSize=None)

Skipped 0 non-data lines
CPU times: user 14.7 s, sys: 1.17 s, total: 15.9 s
Wall time: 15.9 s


In [7]:
#%%time
#xyzi = np.loadtxt(fn, skiprows=11)

In [8]:
%%time
# lets first just remove all the zero data
mask = i != 0.0
i = i[mask]

numFilteredOut = len(x) - len(i)
percent = (float(numFilteredOut) / float(len(x))) * 100.
print("Filtered out %d points of %d (%5.2f%%) intensity equal to zero" % (numFilteredOut, len(x), percent))

x = x[mask]
y = y[mask]
z = z[mask]

print("Now we have %d lines of data" % len(x))

Filtered out 0 points of 9364807 ( 0.00%) intensity equal to zero
Now we have 9364807 lines of data
CPU times: user 144 ms, sys: 182 ms, total: 326 ms
Wall time: 323 ms
Parser   : 222 ms


In [9]:
%%time
x, y, z, mask = neighborFilter(x, y, z, 0.122)
i = i[mask]
print("Now we have %d lines of data" % len(x))

neighborFilter reduces 9364807 points to 9124495 points (240312 filtered, 2.566118 %) using tol: 0.122000
Now we have 9124495 lines of data
CPU times: user 2.7 s, sys: 1.41 s, total: 4.11 s
Wall time: 4.1 s


In [10]:
%%time
# we only want the data that has a decent intesity
meanI = np.mean(i)
stdI = np.std(i)
print("Intensity: max=%5.2f, min=%5.2f, mean=%5.2f, std=%5.2f" % (np.max(i),
                                                              np.min(i),
                                                              meanI,
                                                              stdI))

mask = np.logical_and(i > 0.75, i < 0.85)
i = i[mask]

numFilteredOut = len(x) - len(i)
percent = (float(numFilteredOut) / float(len(x))) * 100.
print("Filtered out %d points of %d (%5.2f%%) via intensity" % (numFilteredOut, len(x), percent))

x = x[mask]
y = y[mask]
z = z[mask]

print("Now we have %d lines of data" % len(x))

Intensity: max= 1.00, min= 0.00, mean= 0.72, std= 0.20
Filtered out 2676627 points of 9124495 (29.33%) via intensity
Now we have 6447868 lines of data
CPU times: user 306 ms, sys: 153 ms, total: 459 ms
Wall time: 457 ms


In [11]:
x_ = np.copy(x)
y_ = np.copy(y)
z_ = np.copy(z)

In [12]:
%%time
orgNum = len(x)
x, y, z =  radialFilter(x, y, z, xOffset, yOffset, radius)
newNum = len(x)
print("radial limit filtered out %d points outside radius %5.2f" % ((orgNum - newNum), radius))
print("Now we have %d lines of data" % len(x))

radial limit filtered out 1509832 points outside radius 45.00
Now we have 4938036 lines of data
CPU times: user 264 ms, sys: 154 ms, total: 418 ms
Wall time: 416 ms


In [13]:
%%time
orgNum = len(x_)

mask = np.power(x_ - xOffset, 2.) + np.power(y_ - yOffset, 2.) < radius**2.
x__ = x_[mask]
y__ = y_[mask]
z__ = z_[mask]

newNum = len(x__)
print("radial limit filtered out %d points outside radius %5.2f" % ((orgNum - newNum), radius))
print("Now we have %d lines of data" % len(x))

radial limit filtered out 1509832 points outside radius 45.00
Now we have 4938036 lines of data
CPU times: user 253 ms, sys: 199 ms, total: 452 ms
Wall time: 451 ms


In [14]:
%%time
z = -z

# z - filter: at this point we should have the
# dish, but with some things the radial filter didn't
# get rid of above or below the dish
zLimit = -80
mask = z > -80
orgNum = len(z)
x = x[mask]
y = y[mask]
z = z[mask]
newNum = len(z)
print("z - limit filtered out %d points below %5.2f" % ((orgNum - newNum), zLimit))

z - limit filtered out 0 points below -80.00
CPU times: user 70.3 ms, sys: 116 ms, total: 187 ms
Wall time: 186 ms


In [15]:
%%time
# x, y, z -> [(x, y, z)]
# for rotation phase
xyz = []
for i in range(len(x)):
    xyz.append((x[i], y[i], z[i]))
xyz = np.array(xyz)

CPU times: user 5.04 s, sys: 576 ms, total: 5.62 s
Wall time: 5.61 s


In [16]:
%%time
xyz_ = np.c_[x, y, z]

CPU times: user 75.7 ms, sys: 76.3 ms, total: 152 ms
Wall time: 151 ms


In [18]:
xyz[:,0]

array([ -7.459617,  -7.449204,  -7.439179, ..., -13.406927, -13.387124,
       -13.366927])

In [19]:
xyz_[:,0]

array([ -7.459617,  -7.449204,  -7.439179, ..., -13.406927, -13.387124,
       -13.366927])

In [21]:
%%time
print("Rotating about Z by %5.2f degrees" % rot)
rotationAboutZdegrees = rot
xyz = rotateXYaboutZ(xyz, rotationAboutZdegrees)

Rotating about Z by 80.00 degrees
CPU times: user 4.07 s, sys: 577 ms, total: 4.65 s
Wall time: 4.64 s


In [25]:
%%time
print("Rotating about Z by %5.2f degrees" % rot)
rotationAboutZdegrees = rot
xyz_newrot = rotateXYaboutZ(xyz_, rotationAboutZdegrees)

Rotating about Z by 80.00 degrees
CPU times: user 297 ms, sys: 280 ms, total: 577 ms
Wall time: 573 ms


In [26]:
xyz_newrot[:,0]

array([ 21.0314266 ,  21.00208927,  20.97382497, ..., -20.24037259,
       -20.21046909, -20.17974873])

In [24]:
xyz[:,0]

array([ 21.0314266 ,  21.00208927,  20.97382497, ..., -20.24037259,
       -20.21046909, -20.17974873])